In [107]:
### Feature Extractor ###

# Import the functions to calculate feature descriptors
from skimage.feature import hog
from skimage.io import imread
import joblib
import cv2
# To read file names
import glob
import os

def convert_to_gray(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return gray

if __name__ == "__main__":
    
    orientations = 9
    pixels_per_cell = (8, 8)
    cells_per_block = (2, 2)
    pos_im_path = "dataset/train/positives"
    neg_im_path = "dataset/train/negatives"
    pos_feat_ph = ("output/positives_features")
    neg_feat_ph = ("output/negatives_features")
        
    #"Calculating the descriptors for the positive samples and saving them"
    for im_path in glob.glob(os.path.join(pos_im_path, "*")):
        im = imread(im_path)
        im = convert_to_gray(im)
        fd = hog(im, orientations, pixels_per_cell, cells_per_block)
        fd_name = os.path.split(im_path)[1].split(".")[0] + ".feat"
        fd_path = os.path.join(pos_feat_ph, fd_name)
        joblib.dump(fd, fd_path)
    print ("Positive features saved in {}".format(pos_feat_ph))

    #"Calculating the descriptors for the negative samples and saving them"
    for im_path in glob.glob(os.path.join(neg_im_path, "*")):
        im = imread(im_path)
        im = convert_to_gray(im)
        fd = hog(im,  orientations, pixels_per_cell, cells_per_block)
        fd_name = os.path.split(im_path)[1].split(".")[0] + ".feat"
        fd_path = os.path.join(neg_feat_ph, fd_name)
        joblib.dump(fd, fd_path)
    print("Negative features saved in {}".format(neg_feat_ph))

Positive features saved in output/positives_features
Negative features saved in output/negatives_features


In [130]:
### Training the classifier (SVM) ###

# Import the functions to calculate feature descriptors
from skimage.feature import hog
from sklearn.svm import LinearSVC
import joblib
import cv2
# To read file names
import glob
import os

if __name__ == "__main__":

    model_path = "output/models"
    pos_feat_path = "output/positives_features"
    neg_feat_path = "output/negatives_features"

    fds = []
    labels = []
    
    # Load the positive features
    for feat_path in glob.glob(os.path.join(pos_feat_path,"*.feat")):
        fd = joblib.load(feat_path)
        fds.append(fd)
        labels.append(1)

    # Load the negative features
    for feat_path in glob.glob(os.path.join(neg_feat_path,"*.feat")):
        fd = joblib.load(feat_path)
        fds.append(fd)
        labels.append(0)

    #"Training a Linear SVM Classifier"
    clf = LinearSVC()
    clf.fit(fds, labels)
    
    model_path = os.path.join("output/models", 'model.pkl')
    joblib.dump(clf, model_path)    
        # Print information about the model
    print("Number of features:", clf.coef_.shape[1])
    print("Coefficients shape:", clf.coef_.shape)
    print("Intercept shape:", clf.intercept_.shape)

    
    print ("Classifier saved")

D:\Programs\Anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Number of features: 93636
Coefficients shape: (1, 93636)
Intercept shape: (1,)


AttributeError: 'LinearSVC' object has no attribute 'n_support_'

In [128]:
### Testing the classifier (SVM) ###

# Import the required modules
from skimage.transform import pyramid_gaussian
from skimage.io import imread
from skimage.feature import hog
import joblib
import cv2
from sklearn.preprocessing import StandardScaler

def sliding_window(image, window_size, step_size):
    '''
    This function returns a patch of the input image `image` of size equal
    to `window_size`. The first image returned top-left co-ordinates (0, 0) 
    and are increment in both x and y directions by the `step_size` supplied.
    So, the input parameters are -
    * `image` - Input Image
    * `window_size` - Size of Sliding Window
    * `step_size` - Incremented Size of Window

    The function returns a tuple -
    (x, y, im_window)
    where
    * x is the top-left x co-ordinate
    * y is the top-left y co-ordinate
    * im_window is the sliding window image
    '''
    for y in range(0, image.shape[0], step_size[1]):
        for x in range(0, image.shape[1], step_size[0]):
            yield (x, y, image[y:y + window_size[1], x:x + window_size[0]])

if __name__ == "__main__":

    # Read the image
    img = imread("dataset/test/Golden-Delicious_021_jpg.rf.87337bfc25366d0027bbd3ddfa4bf26a.jpg")

    im = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    fd = hog(im, orientations, pixels_per_cell, cells_per_block)
    fd = np.array(fd).reshape(1, -1)  # Reshape to 2D array with one row
    pred = clf.predict(fd)
    if pred == 1:
        print ("found")
    else:
        print("not found")
    
    min_wdw_sz = (100, 40)
    step_size = (10, 10)
    downscale = 1.25
    visualize_det = "store_true"
    orientations = 9
    pixels_per_cell = (8, 8)
    cells_per_block = (2, 2)

    # Load the classifier
    clf = joblib.load("model.pkl")

    # List to store the detections
    detections = []
    # The current scale of the image
    scale = 0
    # Downscale the image and iterate
    for im_scaled in pyramid_gaussian(im, downscale=downscale):
        # This list contains detections at the current scale
        cd = []
        # If the width or height of the scaled image is less than
        # the width or height of the window, then end the iterations.
        if im_scaled.shape[0] < min_wdw_sz[1] or im_scaled.shape[1] < min_wdw_sz[0]:
            break
        for (x, y, im_window) in sliding_window(im_scaled, min_wdw_sz, step_size):
            if im_window.shape[0] != min_wdw_sz[1] or im_window.shape[1] != min_wdw_sz[0]:
                continue
            # Calculate the HOG features
            print(im_window)
            fd = hog(im_window, orientations, pixels_per_cell, cells_per_block)
            fd = np.array(fd).reshape(1, -1)  # Reshape to 2D array with one row
            pred = clf.predict(fd)
            if pred == 1:
                print ("Detection:: Location -> ({}, {})".format(x, y))
                print ("Scale ->  {} | Confidence Score {} \n".format(scale,clf.decision_function(fd)))
                detections.append((x, y, clf.decision_function(fd),
                    int(min_wdw_sz[0]*(downscale**scale)),
                    int(min_wdw_sz[1]*(downscale**scale))))
                cd.append(detections[-1])
        # Mo#ve the the next scale
        scale+=1
    # Display the results before performing NMS
    clone = im.copy()
    for (x_tl, y_tl, _, w, h) in detections:
        # Draw the detections
        cv2.rectangle(im, (x_tl, y_tl), (x_tl+w, y_tl+h), (0, 0, 0), thickness=2)
    cv2.imshow("Raw Detections before NMS", im)
    cv2.waitKey()

    ## Perform Non Maxima Suppression
    #detections = nms(detections, threshold)

    ## Display the results after performing NMS
    #for (x_tl, y_tl, _, w, h) in detections:
    #    # Draw the detections
    #    cv2.rectangle(clone, (x_tl, y_tl), (x_tl+w,y_tl+h), (0, 0, 0), thickness=2)
    #cv2.imshow("Final Detections after applying NMS", clone)
    #cv2.waitKey()


not found
[[0.24313725 0.24705882 0.25098039 ... 0.72941176 0.76078431 0.75294118]
 [0.23137255 0.23529412 0.24313725 ... 0.74117647 0.76470588 0.75294118]
 [0.21960784 0.22352941 0.22745098 ... 0.75686275 0.77254902 0.75686275]
 ...
 [0.39215686 0.35686275 0.3254902  ... 0.44313725 0.44313725 0.44705882]
 [0.35294118 0.32156863 0.29411765 ... 0.39607843 0.39215686 0.39607843]
 [0.3254902  0.30196078 0.27843137 ... 0.35686275 0.35686275 0.36078431]]


ValueError: X has 1584 features, but LinearSVC is expecting 93636 features as input.